In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.express as px
from pyvis.network import Network
import networkx as nx
from db_inference import simple_sql_db
import prot_db.bigtable_constants as btc

from google.cloud.bigtable.row_set import RowSet
from google.cloud.bigtable import row_filters

import sqlite3


from utils import notebook_util
notebook_util.disp_notebook_full_width()

In [ ]:
seq_db = simple_sql_db.SequenceSqlDb()
sql_db = simple_sql_db.SimpleSqlDb()
table = btc.get_table(cloud=True)

In [ ]:
TGT_ID = "e32c8417a22fee8216"

In [ ]:
def get_row_keys(p30_id):
    row_keys = []
    tgt_p100s = [row['p100'] for row in sql_db.get_p30_cluster_members(TGT_ID)]
    for pid in tgt_p100s:
        aa_seq = seq_db.get_sequence(pid)
        if aa_seq.endswith("*"):
            aa_seq = aa_seq[:-1]
        row_key = btc.row_key(aa_seq)
        row_keys.append(row_key)
    return row_keys

In [ ]:
%%time

row_keys = get_row_keys(TGT_ID)
print(len(row_keys), "proteins")

row_filter = row_filters.CellsColumnLimitFilter(1)

row_set = RowSet()
for row_key in row_keys:
    row_set.add_row_key(row_key)
    
    
rows = table.read_rows(row_set=row_set, filter_=row_filter)

In [ ]:
def parse_annotation(anno):
    parsed = {}
    for line in anno.split(";"):
        k, v = line.split("=")
        parsed[k] = v
    return parsed


sources = []
annotations = []

for row in rows:
    sources.append(btc.get_mgnify_study_to_analysis(row))
    annotations.append(btc.get_annotations_from_row(row))

    
annotations = [parse_annotation(anno) for anno in annotations if anno is not None]
    

In [ ]:
pd.DataFrame(annotations)

In [ ]:
import urllib.parse

urllib.parse.unquote(annotations[0]['eggnog'])

In [ ]:
annotations = [btc.get_annotations_from_row(row) for row in rows]
print(len(annotations))

print(annotations[-1])

In [ ]:
2bbcdfad92c83273dc -> # https://pfam.xfam.org/family/PF04359

2df6766ad7d9fe5716 -> # cas2



In [ ]:
pd.DataFrame(parsed)['pfam'].value_counts()

In [ ]:
pd.DataFrame(parsed)